In [5]:
from huggingface_hub import notebook_login
notebook_login()


# Install and import


In [1]:
!pip install imblearn

In [2]:
!pip install datasets transformers huggingface_hub


In [3]:
!pip install evaluate

  Using cached evaluate-0.4.2-py3-none-any.whl.metadata (9.3 kB)
Using cached evaluate-0.4.2-py3-none-any.whl (84 kB)


In [4]:
import numpy as np
from datasets import load_metric
from datasets import load_dataset

# Dataset


## abrindo dataset jvanz/portuguese_sentiment_analysis

In [6]:
dataset = load_dataset("jvanz/portuguese_sentiment_analysis")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/1789621 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/149136 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/447405 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['polarity', 'review_text', 'review_text_processed'],
        num_rows: 1789621
    })
    validation: Dataset({
        features: ['polarity', 'review_text', 'review_text_processed'],
        num_rows: 149136
    })
    test: Dataset({
        features: ['polarity', 'review_text', 'review_text_processed'],
        num_rows: 447405
    })
})

In [8]:
dataset = dataset.remove_columns("review_text")


In [9]:
dataset["train"].num_rows

1789621

In [10]:
small_train_dataset = dataset["train"].shuffle().select([i for i in list(range(50000))])
small_test_dataset = dataset["test"].shuffle().select([i for i in list(range(10000))])


In [11]:
def count_classes(dataset):
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    for item in dataset:
        if item['polarity'] == 0:
            negative_count += 1
        elif item['polarity'] == 1:
            positive_count += 1
        elif item['polarity']==2:
            neutral_count += 1
    return positive_count, negative_count, neutral_count
def show_qnt_classes(dataset, text):
  positive_count, negative_count, neutral_count = count_classes(dataset)
  print(f"({text}) Quantidade de classes positivas: {positive_count}")
  print(f"({text}) Quantidade de classes negativas: {negative_count}")
  print(f"({text}) Quantidade de classes neutras: {neutral_count}")

In [12]:
show_qnt_classes(small_train_dataset, 'train')
show_qnt_classes(small_test_dataset, 'test')

(train) Quantidade de classes positivas: 41352
(train) Quantidade de classes negativas: 8648
(train) Quantidade de classes neutras: 0
(test) Quantidade de classes positivas: 8225
(test) Quantidade de classes negativas: 1775
(test) Quantidade de classes neutras: 0


In [13]:
# pela discrepancia ser muito alta, irei realizar o undersampling
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

def apply_undersampling(dataset, target_column_name):
    features = [example['review_text_processed'] for example in dataset]
    labels = np.array([example[target_column_name] for example in dataset])

    undersampler = RandomUnderSampler(random_state=42)
    features_resampled, labels_resampled = undersampler.fit_resample(np.array(features).reshape(-1, 1), labels)

    balanced_dataset = [{'review_text_processed': features_resampled[i][0], target_column_name: labels_resampled[i]} for i in range(len(features_resampled))]

    return balanced_dataset

In [14]:
small_train_dataset = apply_undersampling(small_train_dataset, 'polarity')
small_test_dataset = apply_undersampling(small_test_dataset, 'polarity')

show_qnt_classes(small_train_dataset, 'train')
show_qnt_classes(small_test_dataset, 'test')

(train) Quantidade de classes positivas: 8648
(train) Quantidade de classes negativas: 8648
(train) Quantidade de classes neutras: 0
(test) Quantidade de classes positivas: 1775
(test) Quantidade de classes negativas: 1775
(test) Quantidade de classes neutras: 0


In [15]:
dataset['test'][0]

{'polarity': 1.0,
 'review_text_processed': 'confesso que, para alem de o filme ser maravilhoso e original como qualquer outras coisa que o manoel tenha realizado, o que mais me cativa mesmo e uma cena de roubo noturno, mostrando que, por detras de toda a paz e ocio criativo burgues, existem forcas que tornem mais do que justificaveis o belo titulo deste filme! (wpc>)'}

In [16]:
small_test_dataset[:10]

[{'review_text_processed': 'a descricao  do produto  e  samsung, fiz a compra e veio um produto de uma outra marca.',
  'polarity': 0.0},
 {'review_text_processed': 'lixo nao pega nada esse aplicativo lixoooooo',
  'polarity': 0.0},
 {'review_text_processed': 'um videoclipe de 1 hora e 40 minutos.',
  'polarity': 0.0},
 {'review_text_processed': 'muito ruim o layout, esta lento. e melhor a versao anterior!',
  'polarity': 0.0},
 {'review_text_processed': 'nao e nem de longe algo a ser considerado bom!bem mediano... daqueles mata-tempo, quando vc quer distrair a mente!',
  'polarity': 0.0},
 {'review_text_processed': 'e uma falta de respeito........ quero o produto  entregue em minha residenci.a',
  'polarity': 0.0},
 {'review_text_processed': 'ja procurei as taxas do recargaplay e nao encontro ! estou usando o celcoin mais estou achando muito baixo as porcentagem. 2% de vndas de jogos e % de boletos horriveis. estou com uma movimentacao boa, e estou procurando lucros melhores. alguem p

In [17]:
small_test_dataset

[{'review_text_processed': 'a descricao  do produto  e  samsung, fiz a compra e veio um produto de uma outra marca.',
  'polarity': 0.0},
 {'review_text_processed': 'lixo nao pega nada esse aplicativo lixoooooo',
  'polarity': 0.0},
 {'review_text_processed': 'um videoclipe de 1 hora e 40 minutos.',
  'polarity': 0.0},
 {'review_text_processed': 'muito ruim o layout, esta lento. e melhor a versao anterior!',
  'polarity': 0.0},
 {'review_text_processed': 'nao e nem de longe algo a ser considerado bom!bem mediano... daqueles mata-tempo, quando vc quer distrair a mente!',
  'polarity': 0.0},
 {'review_text_processed': 'e uma falta de respeito........ quero o produto  entregue em minha residenci.a',
  'polarity': 0.0},
 {'review_text_processed': 'ja procurei as taxas do recargaplay e nao encontro ! estou usando o celcoin mais estou achando muito baixo as porcentagem. 2% de vndas de jogos e % de boletos horriveis. estou com uma movimentacao boa, e estou procurando lucros melhores. alguem p

## adicionando mais dados

In [20]:
import pandas as pd
data_df = pd.read_csv("/content/Tweets_Mg.csv")
data_df = data_df.drop(['Unnamed: 0', 'Created At', 'Geo Coordinates.latitude', 'Geo Coordinates.longitude', 'User Location', 'Username', 'User Screen Name', 'Retweet Count', 'Observação'], axis=1)
for i in range(10, 25, 1):
  data_df = data_df.drop(f"Unnamed: {i}", axis=1)

In [21]:
label2id = {'Negativo': 0.0, 'Neutro': 2.0, 'Positivo': 1.0}
for i in range(len(data_df)):
  data_df.at[i, 'Classificacao'] = label2id[data_df.at[i, 'Classificacao']]
  data_df.at[i, 'Classificacao'] = int(data_df.at[i, 'Classificacao'])
  data_df.at[i, 'Text'] = data_df.at[i, 'Text'].lower()

data_df.rename(columns={'Text': 'review_text_processed', 'Classificacao': 'polarity'}, inplace=True)

In [22]:
# apagar os positivos
data_df = data_df[data_df['polarity'] != 1]

In [22]:
import random
random.seed(42)

def split_data(data, train_ratio=0.7, test_ratio=0.2, random_seed=None):
    """
    Splits a list of dictionaries into train, test, and validation sets.

    Args:
    - data: List of dictionaries representing data points.
    - train_ratio: Ratio of data to be allocated for training.
    - test_ratio: Ratio of data to be allocated for testing.
    - val_ratio: Ratio of data to be allocated for validation.
    - random_seed: Seed for random number generator for reproducibility.

    Returns:
    - train_data: List of dictionaries for training.
    - test_data: List of dictionaries for testing.
    - val_data: List of dictionaries for validation.
    """
    # Check if the ratios sum up to 1
    if not train_ratio + test_ratio <= 1:
        raise ValueError(
            "The sum of train_ratio, test_ratio must be less or equal to 1.")

    # Set random seed for reproducibility
    if random_seed is not None:
        random.seed(random_seed)

    # Shuffle the data
    data = data.sample(frac=1).reset_index(drop=True)

    # Calculate split points
    num_data = len(data)
    train_split = int(train_ratio * num_data)
    test_split = int((train_ratio + test_ratio) * num_data)

    # Split the data
    train_data = data[:train_split]
    test_data = data[train_split:]

    return train_data, test_data


train_data, test_data = split_data(
    data_df, train_ratio=0.8, test_ratio=0.2)

In [23]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [24]:
train_dataset

Dataset({
    features: ['review_text_processed', 'polarity'],
    num_rows: 6559
})

In [25]:
show_qnt_classes(train_dataset, 'train')
show_qnt_classes(test_dataset, 'train')


(train) Quantidade de classes positivas: 2635
(train) Quantidade de classes negativas: 1974
(train) Quantidade de classes neutras: 1950
(train) Quantidade de classes positivas: 665
(train) Quantidade de classes negativas: 472
(train) Quantidade de classes neutras: 503


In [26]:
def combine_pandas_dataset(data1, data2):
  df_existing = pd.DataFrame(data1)
  df_combined = pd.concat([df_existing, data2], ignore_index=True)
  return df_combined

In [27]:
from datasets import load_dataset, concatenate_datasets
df_existing = pd.DataFrame(small_train_dataset)
df_combined = pd.concat([df_existing, train_data], ignore_index=True)
small_train_dataset = Dataset.from_pandas(df_combined)

df_existing = pd.DataFrame(small_test_dataset)
df_combined = pd.concat([df_existing, test_data], ignore_index=True)
small_test_dataset = Dataset.from_pandas(df_combined)
#small_train_dataset = concatenate_datasets([small_train_dataset, train_dataset])
#small_test_dataset = concatenate_datasets([small_test_dataset, test_dataset])

In [28]:
small_test_dataset

Dataset({
    features: ['review_text_processed', 'polarity'],
    num_rows: 5190
})

In [29]:
show_qnt_classes(small_train_dataset, 'train')
show_qnt_classes(small_test_dataset, 'train')


(train) Quantidade de classes positivas: 11283
(train) Quantidade de classes negativas: 10622
(train) Quantidade de classes neutras: 1950
(train) Quantidade de classes positivas: 2440
(train) Quantidade de classes negativas: 2247
(train) Quantidade de classes neutras: 503


## abrindo from json

In [30]:
jsonFile = pd.read_json("/content/fake-data-chatgpt-sentiment-analysis.json")

jsonFile.head()

,polarity,review_text_processed
0,0,Estou decepcionado com o resultado do projeto.
1,0,A viagem foi cansativa e estressante.
2,0,Não consigo lidar com toda essa pressão.
3,0,O serviço ao cliente foi terrível.
4,0,Sinto-me triste e sozinho ultimamente.


In [31]:
train_data_chat, test_data_chat = split_data(
    jsonFile, train_ratio=0.8, test_ratio=0.2)

In [32]:
train_data_chat.head()

,polarity,review_text_processed
0,0,Estou insatisfeito com o desempenho do serviço.
1,0,O serviço de transporte foi ineficiente e demo...
2,0,O produto não correspondeu ao que foi prometido.
3,0,O atendimento ao cliente foi desastroso e não ...
4,2,A reunião transcorreu sem incidentes.


In [33]:
small_train_dataset = Dataset.from_pandas(combine_pandas_dataset(small_train_dataset, train_data_chat))
small_test_dataset = Dataset.from_pandas(combine_pandas_dataset(small_test_dataset, test_data_chat))

In [34]:
show_qnt_classes(small_train_dataset, 'train')
show_qnt_classes(small_test_dataset, 'train')

(train) Quantidade de classes positivas: 11283
(train) Quantidade de classes negativas: 10793
(train) Quantidade de classes neutras: 2047
(train) Quantidade de classes positivas: 2440
(train) Quantidade de classes negativas: 2295
(train) Quantidade de classes neutras: 522


# Config model

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = "distilbert-base-uncased"#"lipaoMai/sentiment-model-portuguese-distilbert-pos-neg" #"distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [36]:
def preprocess_function(examples):
    # Tokenizar o texto
    tokenized = tokenizer(examples['review_text_processed'], truncation=True, padding=True)
    # Converter labels para inteiros
    tokenized['labels'] = [int(polarity) for polarity in examples['polarity']]
    return tokenized

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/24123 [00:00<?, ? examples/s]

Map:   0%|          | 0/5257 [00:00<?, ? examples/s]

In [37]:
tokenized_train

Dataset({
    features: ['review_text_processed', 'polarity', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 24123
})

In [38]:
tokenized_train = tokenized_train.remove_columns("polarity")
tokenized_test = tokenized_test.remove_columns("polarity")

In [39]:
tokenized_train = tokenized_train.remove_columns("review_text_processed")
tokenized_test = tokenized_test.remove_columns("review_text_processed")

In [40]:
tokenized_train.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None)}

# FineTuning

In [41]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [43]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    #acc = (preds == p.label_ids).mean()
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [44]:
from transformers import TrainingArguments, Trainer

repo_name = "BERT-sentiment-analysis-portuguese-with-undersampling-v2"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.1,
    warmup_ratio=0.9,
    eval_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_steps=5,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.412500,0.426949,0.829180,0.828964,0.833935,0.829180
2,0.328500,0.308220,0.873502,0.873314,0.873677,0.873502


TrainOutput(global_step=3016, training_loss=0.5082380548218359, metrics={'train_runtime': 2345.2724, 'train_samples_per_second': 20.572, 'train_steps_per_second': 1.286, 'total_flos': 6391136090363904.0, 'train_loss': 0.5082380548218359, 'epoch': 2.0})

In [46]:
trainer.evaluate()

{'eval_loss': 0.30821970105171204,
 'eval_accuracy': 0.8735019973368842,
 'eval_f1': 0.8733141599070333,
 'eval_precision': 0.8736773353227919,
 'eval_recall': 0.8735019973368842,
 'eval_runtime': 71.86,
 'eval_samples_per_second': 73.156,
 'eval_steps_per_second': 4.578,
 'epoch': 2.0}

In [47]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/lipaoMai/BERT-sentiment-analysis-portuguese-with-undersampling-v2/commit/3af67a189dcd6b55fd5ab69d1bfb4213e20f408a', commit_message='End of training', commit_description='', oid='3af67a189dcd6b55fd5ab69d1bfb4213e20f408a', pr_url=None, pr_revision=None, pr_num=None)